[toc]

# Tensorflow tfrecord

## 写入 tfrecord 文件

In [4]:
import tensorflow as tf
import numpy as np

tfrecords_filename = './train.tfrecords'
writer = tf.python_io.TFRecordWriter(tfrecords_filename)  # 创建.tfrecord文件，准备写入

for i in range(100):
    # 模拟图像数据
    img_raw = np.random.random_integers(0, 255,
                                        size=(7, 30))  # 创建7*30，取值在0-255之间随机数组
    img_raw = img_raw.tostring(
    )  # array 类型的数据要转换为 string 然后用 tf.train.ByteList
    features = tf.train.Features(
        feature={
            'label':
            tf.train.Feature(int64_list=tf.train.Int64List(value=[i])),
            'img_raw':
            tf.train.Feature(bytes_list=tf.train.BytesList(value=[img_raw]))
        })
    example = tf.train.Example(features=features)
    writer.write(example.SerializeToString())

writer.close()

/opt/anaconda3/envs/tars/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: This function is deprecated. Please call randint(0, 255 + 1) instead
  if __name__ == '__main__':
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  # Remove the CWD from sys.path while we load stuff.


## 读取

### old 方式

In [2]:
tfrecords_filename = "train.tfrecords"
filename_queue = tf.train.string_input_producer([tfrecords_filename], )  #读入流中
reader = tf.TFRecordReader()
_, serialized_example = reader.read(filename_queue)  #返回文件名和文件


def _parse_record(example_photo):
    features = {
        'label': tf.FixedLenFeature([], tf.int64),
        'img_raw': tf.FixedLenFeature([], tf.string),
    }
    parsed_features = tf.parse_single_example(
        example_photo, features)  #取出包含image和label的feature对象
    image = tf.decode_raw(parsed_features['img_raw'], tf.int64)
    image = tf.reshape(image, [7, 30])
    label = tf.cast(parsed_features['label'], tf.int64)
    return image, label


image, label = _parse_record(serialized_example)
with tf.Session() as sess:  #开始一个会话
    coord = tf.train.Coordinator()  # 创建一个协调器，管理线程
    threads = tf.train.start_queue_runners(
        coord=coord)  # 启动QueueRunner, 此时文件名队列已经进队。
    for i in range(3):
        example, l = sess.run([image, label])  #在会话中取出image和label
        print(example, l)
    coord.request_stop()
    coord.join(threads)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.TFRecordDataset`.
Instructions for updating:
To constru

### new method

In [4]:
# _*_coding:utf-8_*_
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

input_file = "train.tfrecords"

def _parse_record(example_photo):
    features={
       'label': tf.FixedLenFeature([], tf.int64),
       'img_raw' : tf.FixedLenFeature([], tf.string),
   }
    parsed_features = tf.parse_single_example(example_photo, features)  #取出包含image和label的feature对象
    image = tf.decode_raw(parsed_features['img_raw'], tf.int64)
    image = tf.reshape(image, [7,30])
    label = tf.cast(parsed_features['label'], tf.int64)
    return image, label

def read_test(input_file):
    # 用dataset读取TFRecords文件
    dataset = tf.data.TFRecordDataset(input_file)
    dataset = dataset.map(_parse_record)
    iterator = dataset.make_one_shot_iterator()
    next_batch = iterator.get_next() # 这个是一个数据，相当于 batchsize=1
    return next_batch

next_batch = read_test(input_file)

with tf.Session() as sess:
    for i in range(3):
        features = sess.run(next_batch)
        print(features)

/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorflow/python/frame

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
(array([[212, 150, 186, 203, 231,  32, 159, 103, 216,  74, 255,  38,  64,
        119,  20,  44, 100, 188, 212, 104,  43, 163,   5, 229,  43, 167,
         85, 233, 191,  83],
       [122, 230, 243,  24, 152,  86, 122, 234,  85, 227, 188, 125, 165,
        160, 133, 129, 246, 145, 225, 214,  75, 110,  30, 133,  81, 181,
         78,  24,  26, 123],
       [ 90,  10, 150, 205,   8, 212, 167, 206,  62, 190, 145,  75, 139,
        196,  34, 130, 148, 208, 118, 253, 194, 171, 217, 188,  91,  13,
        117, 230, 191, 231],
       [ 17,  63,  22,  50, 150, 251, 185, 112, 103, 210,  75, 195, 243,
         11, 166, 209,  54, 122, 250,  18, 182, 221, 239, 236,   5, 212,
         26, 167, 116,  94],
       [165, 231, 252, 205,  19, 130,  70, 190, 

/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/tars/lib/python3.7/site-packages/te

#### 制作自己的数据集

一般我们使用 tfrecord 是结合 tf.estimator 或者 keras 来使用的，不需要我们自己读取。但是，有时我们还是会遇到需要自己从 tfrecord 中读取并喂入模型的情况。

我们希望我们可以生成一个 `dataset`，然后用下面的形式遍历

```py
for batch in dataset:
    output = model(batch)
```

利用 new method，我们可以制作自己的数据集。

In [13]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

input_file = "train.tfrecords"

def _parse_record(example_photo):
    features={
       'label': tf.FixedLenFeature([], tf.int64),
       'img_raw' : tf.FixedLenFeature([], tf.string),
   }
    parsed_features = tf.parse_single_example(example_photo, features)  #取出包含image和label的feature对象
    image = tf.decode_raw(parsed_features['img_raw'], tf.int64)
    image = tf.reshape(image, [7,30])
    label = tf.cast(parsed_features['label'], tf.int64)
    return image, label

def get_batch(input_file, sess):
    # 用dataset读取TFRecords文件
    dataset = tf.data.TFRecordDataset(input_file)
    dataset = dataset.map(_parse_record)
    iterator = dataset.make_one_shot_iterator()
    while True:
        try:
            data = sess.run(iterator.get_next())
        except tf.errors.OutOfRangeError:
            break
        else:
            yield data

with tf.Session() as sess:
    i = 0
    dataset = get_batch(input_file, sess) # dataset 是一个生成器
    for image, label in dataset:
        print("\rbatch: {}".format(i), end='')
        i += 1

batch: 99

## 坑

### 1. InvalidArgumentError: Key: my_key. Can't parse serialized Example.

这个主要是读取时的数据形状和写入时的不同导致的。

In [4]:
# _*_coding:utf-8_*_
import tensorflow as tf
import numpy as np
max_seq_len = 40
num_of_class = 10

input_file = '/Users/bytedance/Git/devbox/data/test-data.tsv.tfrecord'
def _parse_record(example_photo):
    features = {
        "input_ids":
            tf.FixedLenFeature([max_seq_len], tf.int64),
        "input_mask":
            tf.FixedLenFeature([max_seq_len], tf.int64),
        "input_labels":
            tf.FixedLenFeature([num_of_class], tf.int64),
    }

    parsed_features = tf.parse_single_example(example_photo, features=features)
    return parsed_features
 
def read_test(input_file):
    # 用dataset读取TFRecords文件
    dataset = tf.data.TFRecordDataset(input_file)
    dataset = dataset.map(_parse_record)
    iterator = dataset.make_one_shot_iterator()
    return iterator

iterator = read_test(input_file)

with tf.Session() as sess:
    features = sess.run(iterator.get_next())
    print(features)


{'input_ids': array([  101, 20662, 11444,  3326, 43267, 48626, 21334, 92425, 11325,
        3025,  3025,   102,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0]), 'input_labels': array([1, 1, 1, 1, 0, 0, 0, 0, 0, 0]), 'input_mask': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])}


# References
1. [tensorflow读取数据-tfrecord格式_几何君的算法天空-CSDN博客_tfrecord](https://blog.csdn.net/happyhorizion/article/details/77894055)
2. [tensorflow TFRecords文件的生成和读取方法 - 知乎](https://zhuanlan.zhihu.com/p/31992460)
3. [tensorflow学习笔记——高效读取数据的方法（TFRecord） - 战争热诚 - 博客园](https://www.cnblogs.com/wj-1314/p/11211333.html)

4.  [TFRecord文件查看包含的所有Features_海涛技术漫谈-CSDN博客](https://blog.csdn.net/zhanht/article/details/100177528)

5. [python - Tensorflow TFRecord: Can't parse serialized example - Stack Overflow](https://stackoverflow.com/questions/53499409/tensorflow-tfrecord-cant-parse-serialized-example)